# Stratification - beating the $1 / \sqrt{N}$ law

The key idea in stratification is to split the data (or domain for integrals) on which we wish to calculate an expectation into strata (i.e. parts). Then, on each of these strata, we calculate the expectation separately, using whatever method is appropriate for the stratum, and which gives us the lowest variance. These expectations are then combined together to get the final value, which has better precision than the usual $1 / \sqrt{N}$ law.

In other words we can achieve better sampling in needed regions by going away from a one-size-fits-all sampling scheme. One way to think about it is that regions with higher variability might need more samples, while not-so-varying regions could make do with less.

Stratification is thus a method for obtaining better estimates by subdividing a sample into more homogeneous strata, and then combining the result from each of these to get the final result. The method especially has its real life application in situations, where sampling is expensive, e.g. poling for elections, sampling customers, and patient data analysis.

For more information see P. R. Bevington: page 75-78

***

### Authors: 
- Troels C. Petersen (Niels Bohr Institute)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats

Define the parameters of the program:

In [ ]:
save_plots = False         # Determining if plots are saved or not
r = np.random
r.seed(42)

# Define short hand function:
Y = lambda x: x/(x**2+1.0);

# Set parameters:
N = 10000         # Number of points
Ns = 10           # Number of strata 
Ntry = 1000       # Number of points we use for finding the Std. in each strata
Nrep = 1000       # Number of times we repeat the stratification experiment

xmin =  0
xmax = 10
step = (xmax - xmin) / Ns

Imc = np.zeros(Nrep)    # Results from "normal" MC sampling (i.e. unstratified)
Is  = np.zeros(Nrep)    # Results from stratified sampling with equal sampling in each strata (better)
Is2 = np.zeros(Nrep)    # Results from stratified sampling with sampling according to Std. in strata (best)

In [ ]:
# If we wanted to estimate the integral, then we would use the below (not used in this exercise):
intY = lambda x: np.log(x**2 + 1.0)/2.0;

# Analytic solution :
Ic = intY(xmax)-intY(xmin)

## Plot the distribution and its stratification:

In [ ]:
plt.figure(figsize=[12,6])

## Ploting the original functions h(x)
plt.subplot(1,2,1)
x = np.linspace(0,10,100)
plt.plot(x, Y(x), label=u'$h(x) = x/(x^2+1)$')
plt.legend(loc="upper right", fontsize=18)
plt.ylim(0.0,None)
for j in range(Ns+1):
    plt.axvline(xmin + j*step, 0, 1, color='r', alpha=0.2)

# Plotting the values obtained from inserting uniformily into h(x)
plt.subplot(1,2,2)
Utry = np.random.uniform(low=xmin, high=xmax, size=Ntry)      # Ntry random uniform numbers
Ytry = Y(Utry)                                                # h(x) value of the above numbers
plt.hist(Ytry, bins=50, range=(0.0, 0.5));

## Idea of stratification:

We want to find the mean of $h(x)$ given some (fixed) number of samplings of $x$. We can of course just take a lot of values of $x$ and calculate values of $h(x)$ and then take the mean of these. This is the usual Monte Carlo method that we use.

But when $x$ is high, the standard deviation (Std) is not very large in $h(x)$, and so we can get a better estimate from:<br>
1. Calculating the mean in each strata (using an equal number of samplings in each) and then combining these means.
2. Use a different number of samplings in each strata, proportinal to the Std (since small Std requires small number of samplings).

Thus, instead of taking the mean of $N$ samples, we break the interval into $M$ strata and take $n_j$ samples for each strata $j$, such that $N=\sum_j n_j$.

If we defining the mean for each strata as $\hat{\mu}_j = \frac{1}{n_j} \sum_{x_{ij} \in f_j} h(x_{ij})$, then we can then define the stratified estimator of the overall expectation based on the above means: $\hat{\mu}_s = \sum_j p_j \hat{\mu}_j$, which is an unbiased estimator of $\mu$.

In [ ]:
sigmas = np.zeros(Ns)
Umin = 0 
Umax = step
for strata in np.arange(0, Ns) :
    strata_mask = (Utry >= Umin) & (Utry < Umax)     # Select only the values that are in the current strata
    sigmas[strata] = np.std(Ytry[strata_mask])
    Umin = Umin + step
    Umax = Umin + step

# From the above, calculate the suggested number of samplings to make in each strata:
nums = np.floor(N*sigmas/np.sum(sigmas) + 0.5).astype(int)

# Print results:
np.set_printoptions(precision=3)
print("Sigmas: ", sigmas)        # Std. (i.e. proportionel to uncertainty in mean) in each strata
print("Numbers:", nums)         # Suggested number of samplings in each strata
print("Sum:    ", np.sum(nums))     # Sum of suggestions (should roughly match input size)

## Plot "experimental" process:

Distribution of points from one experiment:

In [ ]:
for k in np.arange(0, Nrep):

    # First, lets do it with mean MC (standard) method: 
    U = np.random.uniform(low=xmin, high=xmax, size=N)
    Imc[k] = (xmax-xmin) * np.mean(Y(U))

    # Next, stratified it in Ns regions with equal (I) and optimal (I2) number in each strata:
    Umin = 0 
    Umax = step
    Ii = 0
    I2i = 0
    for reg in np.arange(0, Ns) :
        x = np.random.uniform(low=Umin, high=Umax, size=N//Ns);
        Ii = Ii + (Umax-Umin)*np.mean(Y(x))
        x2 = np.random.uniform(low=Umin, high=Umax, size=nums[reg]);
        I2i = I2i + (Umax-Umin)*np.mean(Y(x2))
        Umin = Umin + step
        Umax = Umin + step


    Is[k] = Ii
    Is2[k] = I2i

# Resulting Std of the estimates:
print(f"  Standard MC: {np.std(Imc):6.4f}     Stratified:  Equal N: {np.std(Is):6.4f}   Sigma N: {np.std(Is2):6.4f}")

In [ ]:
# Plotting the distribution of results from repeated "experiments":
Nbins = 80
plt.hist(Imc, bins=Nbins, range=(2.27,2.35), histtype='stepfilled', label=u'Normal MC', alpha=0.5)
plt.hist(Is,  bins=Nbins, range=(2.27,2.35), histtype='stepfilled', label=u'Stratified (equal)', alpha=0.4)
plt.hist(Is2, bins=Nbins, range=(2.27,2.35), histtype='stepfilled', label=u'Stratified (sigma)', alpha=0.3)
plt.legend();

***

# Questions:

0. First acquaint yourself with the program, and make sure that you understand what e.g. the parameters `N`, `Ntry`, and `Nrep` refer to!
Then, run the program, and take a look at the results. Make sure you understand both the plots and the numbers printed.

1. How large is the gain in precision when going from MC to equal sampling strata? And when going on to sigma sampling strata?

2. Imagine that you were sampling 100 times from three Gaussian distributions:
       $G_1(\mu=0, \sigma=1.0)$, $G_2(\mu=0.5, \sigma=1.5)$, and $G_1(\mu=1.0, \sigma=4.0)$<br>
   How would you best sample from this combined (3G) PDF? Repeat the above exercises (using known Stds) and see the effect.

# Learning points:

This is an exercise in stratification, i.e. subdividing a sample into more homogenius subsamples to achieve better sample estimates (e.g. mean).

From the exercise you should:
1. Know the difference between normal (MC) sampling and stratified sampling, both equal size sampling per strata and proportional to the strata Std.
2. Understand why stratified sampling is more efficient, and that the gain depends on the size of the differences between strata.
3. Be able to perform stratified sampling, and to design a stratified sampling strategy for a planned (real life) sampling.